<h3>By <font color="green">Leigh Lin</font></h3>
<br/>
<font color="navy">I have created this notebook to 
</font> 
<br>
<font color="navy">
<ul>
<li>demonstrate how various techniques can improve the score of CNN.(score  0.99657)</li>
<li>show how Tensorboard can be used to visualize the data.</li>
<li>show how to enable GPU at a Window laptop.</li>
</ul>
</font>

In [16]:
import keras
keras.__version__

'2.2.4'

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from sklearn.model_selection import train_test_split

from keras import layers
from keras import models

from keras.callbacks import TensorBoard

from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from os import makedirs
from os.path import exists, join

<font color="navy">Loading up data</font>

In [18]:
train = pd.read_csv('C://projects//python//kaggle//digits-recognizer//data//digit-recognizer//train.csv')
train.head()
test = pd.read_csv('C://projects//python//kaggle//digits-recognizer//data//digit-recognizer//test.csv')

<font color="navy">Setting up train/test data</font>

In [19]:
Y_train = train.loc[:]['label']
Y_train = Y_train.values
X_train = train.iloc[:,1:]
del train 
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [20]:
# Set the random seed
random_seed = 3
batch_size=86  

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val_original = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

<font color="navy">One hot encoding of labels</font>

In [21]:
# Encode labels to one hot vectors (ex : 1 -> [0,1,0,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 10)
Y_val = to_categorical(Y_val_original, num_classes = 10)

<font color="navy">Create a simple CNN model.</font>
<br/>
<font color="navy">Notice that we have not used dropout or batch-normalization.</font>

In [19]:
model = models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size = (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                36928     
__________

In [20]:
epochs=30 
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))
score = model.evaluate(X_val, Y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 37800 samples, validate on 4200 samples
Epoch 1/30
37800/37800 [==============================] - 38s 997us/step - loss: 0.2510 - acc: 0.9208 - val_loss: 0.0837 - val_acc: 0.9750
Epoch 2/30
37800/37800 [==============================] - 35s 938us/step - loss: 0.0590 - acc: 0.9808 - val_loss: 0.2018 - val_acc: 0.9360
Epoch 3/30
37800/37800 [==============================] - 35s 937us/step - loss: 0.0394 - acc: 0.9876 - val_loss: 0.0527 - val_acc: 0.9848
Epoch 4/30
37800/37800 [==============================] - 35s 935us/step - loss: 0.0303 - acc: 0.9904 - val_loss: 0.0436 - val_acc: 0.9879
Epoch 5/30
37800/37800 [==============================] - 37s 981us/step - loss: 0.0223 - acc: 0.9928 - val_loss: 0.0413 - val_acc: 0.9898
Epoch 6/30
37800/37800 [==============================] - 39s 1ms/step - loss: 0.0189 - acc: 0.9940 - val_loss: 0.0617 - val_acc: 0.9845
Epoch 7/30
37800/37800 [==============================] - 38s 1ms/step 

<font color="navy">The Test accuracy: </font><font color="green">0.9876190476190476</font>    
<font color="navy">It is such a simple model. Obviously we can make some improments.
<br/>The improvements we make are 
<ul/><li>Use data augmentation.</li>
<li>Reduce learning rate when a metric has stopped improving.</li>
<li>Use BatchNormalization.</li></ul>
</font>

In [22]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)

datagen.fit(X_train)

In [23]:
log_dir = 'C://projects//python//kaggle//digits-recognizer//logs'
if not exists(log_dir):
    makedirs(log_dir)
    
# save class labels to disk to color data points in TensorBoard accordingly
with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    np.savetxt(f, Y_val_original)

In [24]:
tensorboard = TensorBoard(batch_size=batch_size,
                          log_dir = log_dir,
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=X_val)

In [25]:
# Reduce learning rate when a metric has stopped improving
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.3, 
                                            min_lr=0.000001)

In [26]:
model = models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size = (3, 3), activation='relu', input_shape=(28, 28, 1)))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization()) 
model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization()) 
model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(layers.BatchNormalization()) 
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu', name='features'))
model.add(layers.Dense(10, activation='softmax'))

<font color="navy">I have enabled GPU on my laptop. So the average time for one epoch is </font><font color="green">21</font><font color="navy"> seconds.</font>
<br/>
<font color="navy">My laptop has a GeForce GTX 860M GPU. Window version is Window 8.1. It has an Intel i7 2.5GHz CPU.</font>

<font color="navy">Test if the computer is using </font><font color="red">GPU</font><font color="navy"> version</font>
<font color="navy">If the result is </font><font color="green">True</font><font color="navy">, then you are using GPU.</font>

In [29]:
import tensorflow as tf 
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

True

<font color="navy">
I have enabled GPU following the steps below.
<ol>
<li>Remove all NVIDIA Corporation software from your computer.</li>
<li>Download and re-install NVIDIA driver</li>
<li>Download and install Visual Studio 2019</li>
<li>Download and install CUDA Toolkit for Windows 8.1</li>
<li>Download and unzip cuDNN</li>
<li>Install Tensorflow-gpu (using Anaconda)</li>
</ol>
Of course it is a simplifised version, maybe too simplifised. It is another blog to get all the details out.
</font>

In [28]:
epochs=40 
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (X_val,Y_val),
                              verbose = 1, 
                              steps_per_epoch=X_train.shape[0] // batch_size, 
                              callbacks=[tensorboard,learning_rate_reduction])
score = model.evaluate(X_val, Y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/40
439/439 [==============================] - 22s 49ms/step - loss: 0.6311 - acc: 0.8688 - val_loss: 0.1649 - val_acc: 0.9690
Epoch 2/40
439/439 [==============================] - 21s 47ms/step - loss: 0.2249 - acc: 0.9504 - val_loss: 0.3352 - val_acc: 0.9512
Epoch 3/40
439/439 [==============================] - 21s 47ms/step - loss: 0.1643 - acc: 0.9644 - val_loss: 0.1388 - val_acc: 0.9724
Epoch 4/40
439/439 [==============================] - 21s 47ms/step - loss: 0.1155 - acc: 0.9722 - val_loss: 0.1089 - val_acc: 0.9786
Epoch 5/40
439/439 [==============================] - 21s 47ms/step - loss: 0.1002 - acc: 0.9764 - val_loss: 0.0897 - val_acc: 0.9831
Epoch 6/40
439/439 [==============================] - 21s 47ms/step - loss: 0.0957 - acc: 0.9779 - val_loss: 0.1381 - val_acc: 0.9771
Epoch 7/40
439/439 [==============================] - 21s 47ms/step - loss: 0.0893 - acc: 0.9792 - val_loss: 0.1007 - val_acc: 0.9824
Epoch 8/40
439/439 [==============================] - 21s 47ms

<font color="navy">You can see the test accuracy improved from </font><font color="green">0.9876190476190476</font><font color="navy"> to </font><font color="green">0.9907142857142858.</font>

<font color="navy">Now let's prepare out submission file.</font>

In [14]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [15]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("C://projects//python//kaggle//digits-recognizer//data//digit-recognizer//cnn_mnist_submission.csv",index=False)

<font color="navy">My submission scored </font><font color="green">0.99657</font><font color="navy">, which is pretty good considering I am not using any ensemble methods.</font>

<font color="navy">Now we can use Tensorboard to visualize the learning result.</font>

<font color="navy">I am using Anaconda. So at Anaconda environment, open a new Terminal. Then type the following</font>
<br/>
<font color="black">tensorboard --logdir=C:\projects\python\kaggle\digits-recognizer\logs</font>
<br/>
<font color="navy">Once open, click "allow firewall access"</font>
<font color="navy">Then open a new browser tab, type the location</font>
<br/>
<font color="navy">http://localhost:6006</font>

<font color="navy">Thanks for Yassine Ghouzam of his kernel</font>
<a href="https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6" target="_blank">
Introduction to CNN Keras - Acc 0.997 (top 8%)</a>
<font color="navy">Also inspired from the book <a href="https://www.manning.com/books/deep-learning-with-python">Deep learning with python</a> book by Keras author François Chollet.</font>

<font color="navy">Please </font><font color="green" size=5>upvote</font><font color="navy"> my post if you find is useful</font>